In [ ]:
# Cài đặt thư viện
!pip install protobuf==3.20.3 --quiet
!pip install -q transformers datasets peft torch accelerate bitsandbytes evaluate sacrebleu

# Thiết lập môi trường
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
from tqdm import tqdm
SYSTEM_PROMPT_VI_EN = "You are a professional medical translator. Translate the following medical sentence from Vietnamese to English."
SYSTEM_PROMPT_EN_VI = "You are a professional medical translator. Translate the following medical sentence from English to Vietnamese."

print("Đã khởi động lại sạch sẽ!")

In [4]:
# ===================================================================
# == HUẤN LUYỆN CHIỀU VIỆT -> ANH (VI -> EN) - TRÊN MÁY SẠCH
# ===================================================================

# 1. Tải mô hình và Tokenizer
model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

print("Đang tải mô hình...")
model_vi_en = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto", # Máy sạch thì auto sẽ tự phân bổ hợp lý
)
model_vi_en.config.pad_token_id = tokenizer.pad_token_id

# 2. Chuẩn bị dữ liệu (Lấy 50k câu cho nhanh)
print("Đang tải dữ liệu...")
with open("/kaggle/input/medicaldataset-vlsp/MedicalDataset_VLSP/train.vi.txt", "r", encoding="utf-8") as f:
    src_texts = f.read().strip().split("\n")[:50000]
with open("/kaggle/input/medicaldataset-vlsp/MedicalDataset_VLSP/train.en.txt", "r", encoding="utf-8") as f:
    tgt_texts = f.read().strip().split("\n")[:50000]

dataset = Dataset.from_dict({"src": src_texts, "tgt": tgt_texts})

# 3. Tokenize
def tokenize_func(example):
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": example["src"]}
    ]
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True) + example["tgt"] + tokenizer.eos_token
    tokenized = tokenizer(text, truncation=True, max_length=1024, padding=False)
    
    # Tạo labels (mask phần prompt)
    prompt_only = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    prompt_len = len(tokenizer(prompt_only, truncation=True, max_length=1024)["input_ids"])
    labels = [-100] * prompt_len + tokenized["input_ids"][prompt_len:]
    
    return {"input_ids": tokenized["input_ids"], "labels": labels, "attention_mask": tokenized["attention_mask"]}

tokenized_dataset = dataset.map(tokenize_func, remove_columns=dataset.column_names)

# 4. LoRA & Trainer
lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"], lora_dropout=0.05, task_type="CAUSAL_LM")
model_vi_en = get_peft_model(model_vi_en, lora_config)

training_args = TrainingArguments(
    output_dir="./qwen-medical-translator-vi-en",
    report_to="none",             # Tắt WandB
    dataloader_num_workers=0,     # Tắt đa luồng CPU
    per_device_train_batch_size=4, # Batch 4 là an toàn trên máy sạch
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    logging_steps=50,
    learning_rate=2e-4,
    fp16=True,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    save_strategy="no", # Không lưu checkpoint giữa chừng cho đỡ tốn chỗ
)

trainer = Trainer(
    model=model_vi_en,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_vi_en)
)

print("\n--- Bắt đầu chạy ---")
trainer.train()

model_vi_en.save_pretrained("./qwen-medical-lora-adapter-vi-en")
print("--- Xong! ---")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Đang tải mô hình...


...

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



--- Bắt đầu chạy ---


Step,Training Loss
50,1.467500
100,1.337300
150,1.305600
200,1.281400
250,1.259500
300,1.236500
350,1.256700
400,1.208400
450,1.218400
500,1.234800


--- Xong! ---


# Đánh giá đồng thời cả hai mô hình

In [16]:
# ===================================================================
# == PHẦN 3: ĐÁNH GIÁ KẾT QUẢ
# ===================================================================
import evaluate
from peft import PeftModel
SYSTEM_PROMPT_VI_EN = "You are a professional medical translator. Translate the following medical sentence from Vietnamese to English."
SYSTEM_PROMPT_EN_VI = "You are a professional medical translator. Translate the following medical sentence from English to Vietnamese."

# --- 2. Hàm chung để đánh giá ---
def evaluate_model(adapter_path, system_prompt, test_src_texts, test_tgt_texts):
    print(f"\n--- Đang đánh giá adapter từ: {adapter_path} ---")
    
    # Tải mô hình gốc và gộp với adapter
    base_model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,
        device_map="auto",
    )
    # Tải tokenizer để đảm bảo nó đúng
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model = model.merge_and_unload() # Gộp lại để tăng tốc độ inference
    
    predictions = []
    for text in tqdm(test_src_texts, desc=f"Translating for {adapter_path}"):
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text}
        ]
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
        
        outputs = model.generate(
            **inputs, max_new_tokens=128, num_beams=3, do_sample=False,
            early_stopping=True
            pad_token_id=tokenizer.pad_token_id
        )
        
        # Chỉ decode phần token được tạo ra mới, bỏ qua phần prompt
        output_tokens = outputs[0][len(inputs["input_ids"][0]):]
        translation = tokenizer.decode(output_tokens, skip_special_tokens=True).strip()
        predictions.append(translation)

    # Tính toán BLEU Score
    bleu_metric = evaluate.load("sacrebleu")
    results = bleu_metric.compute(predictions=predictions, references=test_tgt_texts)
    
    print(f"\nKết quả cho {adapter_path}:")
    print(f"  >> BLEU Score: {results['score']:.2f}")
    
    # In ra một vài ví dụ
    print("\nVí dụ kết quả:")
    for i in range(min(2, len(test_src_texts))):
        print(f"  Câu gốc  : {test_src_texts[i]}")
        print(f"  Tham khảo : {test_tgt_texts[i][0]}")
        print(f"  Mô hình dịch: {predictions[i]}")
    
    # Giải phóng bộ nhớ sau mỗi lần đánh giá
    del base_model
    del model
    torch.cuda.empty_cache()


# --- 3. Tải dữ liệu test và thực hiện đánh giá ---
with open("/kaggle/input/medicaldataset-vlsp/MedicalDataset_VLSP/public_test.en.txt", 'r', encoding='utf-8') as f:
    test_src_en = [line.strip() for line in f.readlines()]
with open("/kaggle/input/medicaldataset-vlsp/MedicalDataset_VLSP/public_test.vi.txt", 'r', encoding='utf-8') as f:
    test_src_vi = [line.strip() for line in f.readlines()]

# Đánh giá chiều Anh -> Việt
evaluate_model(
    adapter_path="./qwen-medical-lora-adapter-en-vi",
    system_prompt=SYSTEM_PROMPT_EN_VI,
    test_src_texts=test_src_en[:10],
    test_tgt_texts=[[t] for t in test_src_vi[:10]] # Sacrebleu yêu cầu dạng list của list
)

# Đánh giá chiều Việt -> Anh
evaluate_model(
    adapter_path="./qwen-medical-lora-adapter-vi-en",
    system_prompt=SYSTEM_PROMPT_VI_EN,
    test_src_texts=test_src_vi[:10],
    test_tgt_texts=[[t] for t in test_src_en[:10]]
)


--- Đang đánh giá adapter từ: ./qwen-medical-lora-adapter-en-vi ---


Translating for ./qwen-medical-lora-adapter-en-vi: 100%|██████████| 10/10 [00:18<00:00,  1.81s/it]



Kết quả cho ./qwen-medical-lora-adapter-en-vi:
  >> BLEU Score: 31.21

Ví dụ kết quả:
  Câu gốc  : Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao
  Tham khảo : Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017
  Mô hình dịch: Kiến thức, thực hành trong sử dụng dịch vụ y tế công cộng của người có thẻ bảo hiểm y tế và các yếu tố ảnh hưởng tại thành phố Vũng Tàu, Lào
  Câu gốc  : Describe knowledge, practices in public health service utilization among health insurance card's holders and influencing factors in Vientiane, Lao PDR, 2017.
  Tham khảo : Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 201

Translating for ./qwen-medical-lora-adapter-vi-en: 100%|██████████| 10/10 [00:15<00:00,  1.58s/it]



Kết quả cho ./qwen-medical-lora-adapter-vi-en:
  >> BLEU Score: 7.79

Ví dụ kết quả:
  Câu gốc  : Thực trạng kiến thức và thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố ảnh hưởng tại tỉnh Viêng Chăn, CHDCND Lào, năm 2017
  Tham khảo : Knowledge, practices in public health service utilization among health insurance card’s holders and influencing factors in Vientiane, Lao
  Mô hình dịch: Current situation of knowledge and practice of people with health insurance cards in using medical examination and treatment services at public health facilities and some factors affecting in Vien Chanh province, Lao People's Democratic Republic, 2017
  Câu gốc  : Mô tả thực trạng kiến thức, thực hành của người có thẻ bảo hiểm y tế trong sử dụng dịch vụ khám chữa bệnh ở các cơ sở y tế công và một số yếu tố liên quan tại tỉnh Viêng Chăn, Cộng hoà Dân chủ Nhân dân Lào năm 2017.
  Tham khảo : Describe knowledge, practices in public healt